In [ ]:
# --- IMPORTANT ---
# Paste your Hugging Face Access Token here
from dotenv import load_dotenv
import os

# Load variables from the .env file
load_dotenv()

# Read the key from the environment
HF_TOKEN = os.getenv('HF_TOKEN')
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

print(f"Starting download for {model_id}...")

# This command downloads the model and tokenizer using your token directly
tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=HF_TOKEN,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

print("Model and tokenizer downloaded successfully!")

/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Starting download for meta-llama/Meta-Llama-3-8B-Instruct...


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Model and tokenizer downloaded successfully!


In [ ]:
import torch

# 1. Define the conversation prompt
# This is the specific format Llama 3 expects
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "What are the three most important skills for a software engineer?"},
]

# 2. Prepare the input for the model
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

# 3. Generate the response
# These are special tokens that tell the model when to stop talking
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

# 4. Decode and print the result
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


While there are many essential skills for a software engineer, here are three that I consider crucial:

1. **Problem-solving and debugging**: A software engineer should be able to analyze complex problems, identify the root cause, and develop effective solutions. This involves breaking down complex problems into smaller, manageable parts, identifying patterns, and using logical thinking to debug and troubleshoot issues. Strong problem-solving skills are critical in software development, as they enable engineers to overcome obstacles and deliver high-quality products.
2. **Communication and collaboration**: Software engineering is a team effort, and effective communication is vital for success. A software engineer should be able to clearly explain complex technical concepts to both technical and non-technical stakeholders, as well as collaborate with colleagues to design, develop, and test software. Strong communication skills help to ensure that everyone is on the same page, reducing m

In [ ]:
{"text": "<s>[INST] What is the full name of the university? [/INST] The full name is Presidency University, Bangalore.</s>"}
{"text": "<s>[INST] Where is Presidency University located? [/INST] The main campus is located in Itgalpura, Yelahanka, Bengaluru.</s>"}
{"text": "<s>[INST] What school does the Information Science and Technology program belong to? [/INST] The Information Science and Technology program is part of the School of Computer Science and Engineering (SOCSE).</s>"}

{'text': '<s>[INST] What school does the Information Science and Technology program belong to? [/INST] The Information Science and Technology program is part of the School of Computer Science and Engineering (SOCSE).</s>'}

In [ ]:
# --- 1. Install Necessary Libraries ---
# This cell installs all the tools we need for fine-tuning.
# Note: We are installing specific versions for stability.
!pip install "transformers==4.40.1" "accelerate==0.29.3" "bitsandbytes==0.43.0" "peft==0.10.0" "trl==0.8.6" "datasets" -q

# --- 2. Log in to Hugging Face ---
# This allows Colab to download the gated Llama 3 model.
from huggingface_hub import login
import os

# It's best to store your token as a secret in Colab for security.
# On the left sidebar, click the key icon (Secrets) and add a new secret:
# Name: HF_TOKEN
# Value: your_hugging_face_token_here (starts with hf_...)
from dotenv import load_dotenv
import os

# Load variables from the .env file
load_dotenv()

# Read the key from the environment
HF_TOKEN = os.getenv('HF_TOKEN')
if not HF_TOKEN:
    print("HF_TOKEN secret not found. Please add it to your Colab secrets.")
else:
    login(token=HF_TOKEN)
    print("Successfully logged in to Hugging Face.")

# --- 3. Load the Model and Tokenizer ---
# We'll load the base Llama 3 8B model in a memory-efficient 4-bit format
# using a technique called QLoRA. This is what makes it possible to
# fine-tune such a large model on a single GPU.
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Meta-Llama-3-8B"

# Configuration for loading the model in 4-bit precision
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the tokenizer (the vocabulary) and the model itself
print("Loading base model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto", # Automatically use the GPU
    token=HF_TOKEN
)

# Llama 3 doesn't have a default padding token, so we set it to the end-of-sequence token.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("Base model loaded successfully.")

# --- 4. Load Your Custom Dataset ---
# Upload your 'dataset.jsonl' file to your Colab environment by dragging
# it into the file explorer on the left sidebar.
from datasets import load_dataset

dataset_path = "dataset.jsonl"
try:
    dataset = load_dataset("json", data_files=dataset_path, split="train")
    print("\nCustom dataset loaded successfully:")
    print(dataset[0]) # Print the first example to verify
except FileNotFoundError:
    print(f"\nERROR: '{dataset_path}' not found. Please upload your dataset to Colab.")


# --- 5. Configure the Fine-Tuning (PEFT with LoRA) ---
# This is where we set up "Parameter-Efficient Fine-Tuning" (PEFT)
# using the LoRA technique. This freezes most of the huge model and only
# trains a small number of "adapter" layers, which is incredibly memory-efficient.
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Prepare the model for k-bit training
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,  # The rank of the update matrices. A higher rank means more trainable parameters.
    lora_alpha=32, # A scaling factor for the LoRA weights.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], # The specific layers of the model to adapt.
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply the LoRA configuration to the model
model = get_peft_model(model, lora_config)
print("\nLoRA adapters configured for training.")

# --- 6. Set Up and Run the Training ---
# We'll use the SFTTrainer from the TRL library, which is designed for this exact task.
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir="./results", # Directory to save the training results
    num_train_epochs=3, # Number of times to train on the full dataset
    per_device_train_batch_size=4, # Number of examples to process at once
    gradient_accumulation_steps=2,
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    fp16=True, # Use 16-bit floating point precision for faster training
    logging_steps=10,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    dataset_text_field="text", # The name of the column in our dataset
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_args,
)

# Start the training! This will take some time.
print("\nStarting the fine-tuning process...")
trainer.train()
print("Fine-tuning complete!")

# --- 7. Save Your New Model Adapter ---
# The result of the training is a small "adapter" file that contains
# the new knowledge. We'll save this for later use.
new_model_name = "llama-3-8b-presidency-gpt"
trainer.save_model(new_model_name)
print(f"\nNew model adapter saved to '{new_model_name}'")


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.1 which is incompatible.
Successfully logged in to Hugging Face.
Loading base model and tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


False

===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/lib/python3.12/dist-packages/cv2/../../lib64')}
The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
The following directories listed in your path were found to be non-existent: {PosixPath('https'), PosixPath('//mp.kaggle.net')}
The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-uejwp8gqwjk7 --tunnel_background_sa

/usr/local/lib/python3.12/dist-packages/bitsandbytes/cuda_setup/main.py:183: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
/usr/local/lib/python3.12/dist-packages/bitsandbytes/cuda_setup/main.py:183: UserWarning: /usr/local/lib/python3.12/dist-packages/cv2/../../lib64:/usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0', 'libcudart.so.12.1', 'libcudart.so.12.2'] as expected! Searching further paths...
  warn(msg)


RuntimeError: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues